In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split



In [2]:
def handle_missing_values(df, option : str='mean'):
    """Fill missing values in data frame in place"""
    
    missing_values = df.isna().sum()
    # print(missing_values)

    # Check if there are any missing values
    if missing_values.any():
        print("There are missing values in the dataset.")
        # Display the count of missing values for each column
        # find the columns with missing values
        columns_with_missing_values = df.columns[missing_values > 0]
        print("Missing values per column:")
        print(missing_values[columns_with_missing_values])

        # fix the missing values
        # we can use df.replace(to_replace=' ', value=-1) to replace all missing values with -1
        for column in columns_with_missing_values:

            if option == 'drop':
                df.dropna()
            elif option == 'mean':
                df[column].fillna(df[column].mean(), inplace=True)
            elif option == 'median':
                df[column].fillna(df[column].median(), inplace=True)
            elif option == 'mode':
                df[column].fillna(df[column].mode()[0], inplace=True)
            elif option == 'linear':
                df[column].interpolate(method='linear', limit_direction='forward', inplace=True)
            elif option == 'quadratic':
                df[column].interpolate(method='quadratic', limit_direction='forward', inplace=True)
            elif option == 'cubic':
                df[column].interpolate(method='cubic', limit_direction='forward', inplace=True)
            elif option == 'spline':
                df[column].interpolate(method='spline', order=3, limit_direction='forward', inplace=True)
        
        print("Missing values per column are fixed\n")

    else:
        print("There are no missing values in the dataset.\n")
       


In [3]:
def normalize_numeric_data(df, numeric_cols):
    """Normalize numeric columns in dataframe in place"""

    from sklearn.preprocessing import MinMaxScaler, StandardScaler


    # print(numeric_cols)
    scaler = MinMaxScaler()
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])



In [4]:

def one_hot_encode(df, categorical_cols):
    """One hot encode categorical columns in dataframee"""
    one_hot_encoded_data  = pd.get_dummies(df, columns=categorical_cols)

    categorical_cols = one_hot_encoded_data.select_dtypes(exclude=np.number).columns.tolist()
    label_encode(one_hot_encoded_data, categorical_cols)

    return one_hot_encoded_data 

    
def label_encode(df, categorical_cols):
    """Label encode categorical columns in dataframe in place"""

    # print(categorical_cols)
    
    label_encoders = {}
    for col in categorical_cols:
        label_encoders[col] = LabelEncoder()
        df[col] = label_encoders[col].fit_transform(df[col])
        

In [5]:
def get_top_score_feature_df(X, y, k=15):
    """Get the top k features with the highest score
        Return a new dataframe with the top k features
    """
    # todo: need to implement this function
    from sklearn.feature_selection import SelectKBest, mutual_info_classif

    print ("k:",k)

    selector = SelectKBest(mutual_info_classif, k=k)
    
    # Get the scores
    selector.fit(X, y)
    scores = selector.scores_

    df_scores = pd.DataFrame(scores, columns=["Score"], index=X.columns)
    df_scores = df_scores.sort_values(by="Score", ascending=False)

    # keep the top k features of the df DataFrame
    top_score_feature_X = X[df_scores.index[:k]]
    # top_score_feature_df doesn't have the label column


    # Plot the scores
    # import matplotlib.pyplot as plt

    # plt.figure(figsize=(10, 10))
    # plt.barh(df_scores.index, df_scores["Score"])
    # plt.xlabel('Score')
    # plt.ylabel('Feature')
    # plt.title('Feature Importance')
    # plt.show()
    
    return top_score_feature_X

In [90]:
class CustomLogisticRegression:
    """Custom Logistic Regression

    Parameters
    ----------
    learning_rate : float, default=0.01
        Learning rate for gradient descent.

    num_iterations : int, default=1000
        Number of iterations for gradient descent.

    verbose : bool, default=False
        Print the progress of training if True.

    Attributes
    ----------
    weights : 1d-array
        Weights after fitting the model.

    bias : float
        Bias after fitting the model.


    returns
    -------
    y_pred_class : 1d-array
        Predictions using the trained model.


    """



    def __init__(self, learning_rate=0.01, num_iterations=1000, verbose=False):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.verbose = verbose
        self.weights = None

    def sigmoid(self, z):
        tmp = np.array(z, dtype=np.float64)
        return 1 / (1 + np.exp(-tmp))
    
    def normailze(self, X):
        """Normalize the dataset X"""
        tmp = np.array(X, dtype=np.float64)
        return (tmp - np.mean(tmp, axis=0)) / np.std(tmp, axis=0)

    def fit(self, X, y, early_stop_threshold = 0):
        """Fit the model using training data"""

        X = self.normailze(X)
        ones = np.ones((X.shape[0], 1)) # add a column of ones to X 
        X = np.concatenate((ones, X), axis=1)
        # Initialize the parameters
        num_samples, num_features = X.shape
        # print("num samples: ", num_samples)
        # print("num features: ", num_features)

        self.weights = np.zeros(num_features)
        

        # Gradient descent
        from tqdm import tqdm
        for i in tqdm(range(self.num_iterations)):

            linear_model = np.dot(X, self.weights)      # y hat = Xw
            
            # Predictions using sigmoid function
            y_pred = self.sigmoid(linear_model)

            # Compute error
            error = -1.0 * np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
            
            # Early terminate Gradient Descent if error in the training set becomes < early_stop_threshold
            if error < early_stop_threshold:
                print(f'Early stopping at Iteration: {i}')
                break
        

            # Compute gradients
            dw = (1 / num_samples) * np.dot(X.T, (y_pred - y))
            # print (dw)

            # Update parameters
            self.weights = self.weights - self.learning_rate * dw     # w = w - alpha * dw

          

    def predict(self, X):
        X= self.normailze(X)
        ones = np.ones((X.shape[0], 1)) # add a column of ones to X 
        X = np.concatenate((ones, X), axis=1)
        """Predict using the trained model"""
        linear_model = np.dot(X, self.weights)      # y hat = Xw
        y_pred = self.sigmoid(linear_model)
        
        y_pred_classified = [1 if i >= 0.5 else 0 for i in y_pred]
        # make this into a numpy array
        y_pred_classified = np.array(y_pred_classified)
        
        return y_pred_classified

        
    def performance(self, y_test, y_pred):
        """Compute performance"""

        FP = 0
        TP = 0
        FN = 0
        TN = 0

        for i in range(len(y_pred)):
            if y_pred[i] == 1 and y_test[i] == 0:
                FP += 1

        for i in range(len(y_pred)):
            if y_pred[i] == 1 and y_test[i] == 1:
                TP += 1

        for i in range(len(y_pred)):
            if y_pred[i] == 0 and y_test[i] == 1:
                FN += 1

        for i in range(len(y_pred)):
            if y_pred[i] == 0 and y_test[i] == 0:
                TN += 1

        accuracy = (TP + TN) / len(y_pred)
        sensitivity = TP / max( (TP + FN), 1e-10)
        precision = TP / max( (TP + FP), 1e-10)
        specificity = TN / max( (TN + FP), 1e-10)
        false_discovery_rate = FP / max( (TP + FP), 1e-10)
        f1 = 2 * precision * sensitivity / max((precision + sensitivity), 1e-10)

        return accuracy, sensitivity,specificity, precision, false_discovery_rate, f1
    

    

In [78]:
class CustomAdaBoostClassifier:
    """CustomAdaBoost Classifier

    Parameters
    ----------
    k : int
        the number of hypotheses in the ensemble

    Attributes
    ----------
    hypotheses : list
        List of weak hypotheses.

    alphas : list
        List of weights of the weak hypotheses.

    """

    def __init__(self, k=10, learning_rate=0.01, num_iterations=1000, early_stop_threshold = 0, verbose=False):
        self.k = k
        self.hypotheses = None
        self.alphas = None
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.verbose = verbose
        self.early_stop_threshold = early_stop_threshold
  
    
    
    def Resample(self, X, y, weights):
        """Resample the dataset using the weights"""

        # Sample indices from the range of the length of y with replacement, using the weights
        indices = np.random.choice(X.shape[0], X.shape[0], replace = True, p=weights)


        print (indices)

        # Create new dataset using the sampled indices

        # print (X.shape)
        # print (y.shape)
        y_new = y[indices]
        X_new = X.iloc[indices]

        return X_new, y_new

    def fit(self, X, y):
        """Fit the model using training data"""

        # Initialize the weights
        num_samples = X.shape[0]
        weights = np.ones(num_samples) / num_samples

        hypotheses = []
        alphas = [] # weights of the hypotheses, Z in the pdf

       
        for _ in range(self.k):
            # create a new dataset using the weights
            X, y = self.Resample(X, y, weights)

            # Train a weak hypothesis using the weights
            hypothesis = CustomLogisticRegression(learning_rate=self.learning_rate, 
                                                    num_iterations=self.num_iterations, 
                                                    verbose=self.verbose)

            hypothesis.fit(X, y, early_stop_threshold=self.early_stop_threshold)      
            

            # Compute the error
            y_pred = hypothesis.predict(X)
            print ("y_pred:", y_pred)
            error = 0
            for j in range(num_samples):
                if y_pred[j] != y[j]:
                    error += weights[j]

            # print ("error:", error) 

            if error > 0.5:
                print("error is greater than 0.5")
                continue

            # Update the weights of the samples. Give the correct predicted samples a lower weight
            for i in range(num_samples):
                if y_pred[i] == y[i]:
                    weights[i] *= error / (1 - error)
            weights = weights / np.sum(weights)

            hypotheses.append(hypothesis)
            # Compute weights of the hypothesis
            alpha = np.log((1 - error) / max(error, 1e-10))  # Avoid division by zero
            alphas.append(alpha)


        self.hypotheses = hypotheses

        self.alphas = alphas/np.sum(alphas) # Normalize the weights of the hypotheses
        print("alphas: ", self.alphas)



    def predict_ada(self, X):
        """Predict using the trained model"""

        num_samples = X.shape[0]
        y_pred = np.zeros(num_samples)

        print("hyp 1:", type(self.hypotheses[0].predict(X)))
        print("hyp 2:", type(self.hypotheses[0].predict(X))[0])


        for i in range(len(self.hypotheses)):
            # weighted majority hypothesis     
            y_pred += self.alphas[i] * self.hypotheses[i].predict(X)
            # for j in range(num_samples):
            #     y_pred[j] += self.alphas[i] * self.hypotheses[i].predict(X.iloc[j].values.reshape(1, -1))
            

        y_pred_classified = [1 if i >= 0.5 else 0 for i in y_pred] # classifies the data into 0 or 1
    
        return y_pred_classified
        
    

    def performance(self, y_test, y_pred):
        """Compute performance"""

        FP = 0
        TP = 0
        FN = 0
        TN = 0

        for i in range(len(y_pred)):
            if y_pred[i] == 1 and y_test[i] == 0:
                FP += 1

        for i in range(len(y_pred)):
            if y_pred[i] == 1 and y_test[i] == 1:
                TP += 1

        for i in range(len(y_pred)):
            if y_pred[i] == 0 and y_test[i] == 1:
                FN += 1

        for i in range(len(y_pred)):
            if y_pred[i] == 0 and y_test[i] == 0:
                TN += 1

        accuracy = (TP + TN) / len(y_pred)
        sensitivity = TP / max( (TP + FN), 1e-10)
        precision = TP / max( (TP + FP), 1e-10)
        specificity = TN / max( (TN + FP), 1e-10)
        false_discovery_rate = FP / max( (TP + FP), 1e-10)
        f1 = 2 * precision * sensitivity / max((precision + sensitivity), 1e-10)

        return accuracy, sensitivity,specificity, precision, false_discovery_rate, f1
    



In [8]:
def telco_customer_churn_preprocess(num_cols_keep = 10):
    df = pd.read_csv('./data/Telco Customer Churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')
    # print(df.head())
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
    # errors='coerce' is a good approach to handle non-numeric values by replacing them with NaN.

    df.replace(r'^\s+$', np.nan, regex=True, inplace=True)
    # drop customerID column because this will not help in prediction and cause trouble in one hot encoding
    df.drop('customerID', axis=1, inplace=True)

    label_col = df.columns[-1]


    # Split into feature and label data
    X = df.drop(columns=[label_col])
    y = df[label_col]

    numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
    # Exclude columns with binary data
    numeric_cols = [col for col in numeric_cols if len(X[col].unique()) > 2]
    # print(numeric_cols)

    categorical_cols = X.select_dtypes(exclude=np.number).columns.tolist()
    # add the binary column back to the list of categorical columns
    binary_cols =  [col for col in numeric_cols if len(X[col].unique()) == 2]
    categorical_cols.extend(binary_cols)
    X = one_hot_encode(X, categorical_cols)


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=82)
    # random_state is the seed used by the random number generator
    # keeping the train and test data separate to avoid data leakage

    handle_missing_values(X_train, option='mean')
    handle_missing_values(X_test, option='mean')


    normalize_numeric_data(X_train, numeric_cols)
    normalize_numeric_data(X_test, numeric_cols)

    y_train = LabelEncoder().fit_transform(y_train)
    y_test = LabelEncoder().fit_transform(y_test)

    X_train = get_top_score_feature_df(X_train, y_train, k=num_cols_keep)

    # keep only the top k features in the test data from the train data
    X_test = X_test[X_train.columns]

    return X_train, X_test, y_train, y_test


Credit Card Data

In [21]:
def credit_card_default_preprocessing(num_cols_keep = 25):
    df_credit = pd.read_csv('./data/Credit Card Fraud Detection/creditcard.csv')

    df_credit.replace(r'^\s+$', np.nan, regex=True, inplace=True)

    # take all the positive samples 
    df_credit_pos = df_credit[df_credit['Class'] == 1]
    # randomly sample 20000 of negative samples
    df_credit_neg = df_credit[df_credit['Class'] == 0].sample(n=20000, random_state=82)

    # combine the positive and negative samples
    df_credit = pd.concat([df_credit_pos, df_credit_neg])


    # drop the time column because it is not useful for prediction
    df_credit.drop('Time', axis=1, inplace=True)

    label_col_credit = df_credit.columns[-1]

    # Split into feature and label data
    X = df_credit.drop(columns=[label_col_credit])
    y = df_credit[label_col_credit]

    numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
    # Exclude columns with binary data
    numeric_cols = [col for col in numeric_cols if len(X[col].unique()) > 2]
    # print(numeric_cols)

    categorical_cols = X.select_dtypes(exclude=np.number).columns.tolist()
    # add the binary column back to the list of categorical columns
    binary_cols =  [col for col in numeric_cols if len(X[col].unique()) == 2]
    categorical_cols.extend(binary_cols)
    X = one_hot_encode(X, categorical_cols)


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=82)
    # random_state is the seed used by the random number generator
    # keeping the train and test data separate to avoid data leakage

    handle_missing_values(X_train, option='mean')
    handle_missing_values(X_test, option='mean')


    normalize_numeric_data(X_train, numeric_cols)
    normalize_numeric_data(X_test, numeric_cols)

    y_train = LabelEncoder().fit_transform(y_train)
    y_test = LabelEncoder().fit_transform(y_test)

    X_train = get_top_score_feature_df(X_train, y_train, k=num_cols_keep)

    # keep only the top k features in the test data from the train data
    X_test = X_test[X_train.columns]

    return X_train, X_test, y_train, y_test



Adult data

In [83]:
def adult_income_preprocessing(num_cols_keep=30):
    col_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship',
                 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

    train_data = pd.read_csv('./data/adult/adult.data', names=col_names)
    test_data = pd.read_csv('./data/adult/adult.test', names=col_names, skiprows=1)

    # drop the fnlwgt column because it is not useful for prediction
    train_data.drop('fnlwgt', axis=1, inplace=True)

    # drop the education column because it is redundant with education-num
    train_data.drop('education', axis=1, inplace=True)

    # drop the country column because it is not useful for prediction
    train_data.drop('native-country', axis=1, inplace=True)

    label_col = train_data.columns[-1]

    # Split into feature and label data
    X_train = train_data.drop(columns=[label_col])
    y_train = train_data[label_col]

    X_test = test_data.drop(columns=[label_col])
    y_test = test_data[label_col]

    # modifying values in specific columns
    y_train.replace({' >50K': 1, ' <=50K': 0}, inplace=True)
    y_test.replace({' >50K.': 1, ' <=50K.': 0}, inplace=True)

    
    X_train.replace(' ?', np.nan, inplace=True)
    X_test.replace(' ?', np.nan, inplace=True)

    handle_missing_values(X_train, option='mode')
    handle_missing_values(X_test, option='mode')

    numeric_cols = X_train.select_dtypes(include=np.number).columns.tolist()
    normalize_numeric_data(X_train, numeric_cols)
    normalize_numeric_data(X_test, numeric_cols)
    

    X_train = pd.get_dummies(X_train)
    X_test = pd.get_dummies(X_test)

    y_train = LabelEncoder().fit_transform(y_train)
    y_test = LabelEncoder().fit_transform(y_test)



    X_train = get_top_score_feature_df(X_train, y_train, k=num_cols_keep)
    X_test = X_test[X_train.columns]

    return X_train, X_test, y_train, y_test


In [80]:
X_train, X_test, y_train, y_test = adult_income_preprocessing(num_cols_keep=30)

There are missing values in the dataset.
Missing values per column:
workclass     1836
occupation    1843
dtype: int64
Missing values per column are fixed

There are missing values in the dataset.
Missing values per column:
workclass         963
occupation        966
native-country    274
dtype: int64
Missing values per column are fixed

k: 30


In [81]:
np.unique(y_train)

array([0, 1], dtype=int64)

In [82]:
np.unique(y_test)

array([0, 1], dtype=int64)

In [94]:
def run_logistic_regression(dataset_name):
    np.random.seed(82)

    if dataset_name == 'telco':
        X_train, X_test, y_train, y_test = telco_customer_churn_preprocess()
        

    if dataset_name == 'credit':
        X_train, X_test, y_train, y_test = credit_card_default_preprocessing()
        

    if dataset_name == 'adult':
        X_train, X_test, y_train, y_test = adult_income_preprocessing()


    model = CustomLogisticRegression(learning_rate=0.01, num_iterations=5000, verbose=False)

 
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("Logistic Regression Test")
    accuracy, sensitivity, specificity, precision, false_discovery_rate, f1 = model.performance(y_test, y_pred)
    print("Accuracy: ", accuracy * 100)
    print("Sensitivity: ", sensitivity * 100)
    print("Specificity: ", specificity * 100)
    print("Precision: ", precision * 100)
    print("False Discovery Rate: ", false_discovery_rate * 100)
    print("F1: ", f1 * 100)

    # save this report in a csv file
    report = pd.DataFrame(columns=['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'False Discovery Rate', 'F1'], index=['Logistic Regression Test'])
    report.loc['Logistic Regression Test'] = [accuracy, sensitivity, specificity, precision, false_discovery_rate, f1]
    report.transpose()
    report.to_csv(f'./logistic_regression_{dataset_name}_report.csv')


    y_pred = model.predict(X_train)

    print("Logistic Regression Train")
    accuracy, sensitivity, specificity, precision, false_discovery_rate, f1 = model.performance(y_train, y_pred)
    print("Accuracy: ", accuracy * 100)
    print("Sensitivity: ", sensitivity * 100)
    print("Specificity: ", specificity * 100)
    print("Precision: ", precision * 100)
    print("False Discovery Rate: ", false_discovery_rate * 100)
    print("F1: ", f1 * 100)
          
    report = pd.DataFrame(columns=['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'False Discovery Rate', 'F1'], index=['Logistic Regression Train'])
    report.loc['Logistic Regression Train'] = [accuracy, sensitivity, specificity, precision, false_discovery_rate, f1]
    report.transpose()
    report.to_csv(f'./logistic_regression_{dataset_name}_report.csv', mode='a')





In [48]:
def run_adaboost(dataset_name, boosting_round = 5):
    np.random.seed(82)
    if dataset_name == 'telco':
        X_train, X_test, y_train, y_test = telco_customer_churn_preprocess()
        

    if dataset_name == 'credit':
        X_train, X_test, y_train, y_test = credit_card_default_preprocessing()
        

    if dataset_name == 'adult':
        X_train, X_test, y_train, y_test = adult_income_preprocessing()


    model = CustomAdaBoostClassifier(
        k=boosting_round,
        learning_rate=0.01,
        num_iterations=1000,
        early_stop_threshold=0.5,
        verbose=False,
    )
    model.fit(X_train, y_train)
    y_pred = model.predict_ada(X_test)



    print("Adaboost Test")
    accuracy, sensitivity, specificity, precision, false_discovery_rate, f1 = model.performance(y_test, y_pred)
    print("Accuracy: ", accuracy * 100)
    print("Sensitivity: ", sensitivity * 100)
    print("Specificity: ", specificity * 100)
    print("Precision: ", precision * 100)
    print("False Discovery Rate: ", false_discovery_rate * 100)
    print("F1: ", f1 * 100)

    # save this report in a csv file
    report = pd.DataFrame(columns=['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'False Discovery Rate', 'F1'], index=['Adaboost Test'])
    report.loc['Adaboost Test'] = [accuracy, sensitivity, specificity, precision, false_discovery_rate, f1]
    report.transpose()
    report.to_csv(f'./adaboost_{dataset_name}_report.csv')


    y_pred = model.predict_ada(X_train)

    print("Adaboost Train")
    accuracy, sensitivity, specificity, precision, false_discovery_rate, f1 = model.performance(y_train, y_pred)
    print("Accuracy: ", accuracy * 100)
    print("Sensitivity: ", sensitivity * 100)
    print("Specificity: ", specificity * 100)
    print("Precision: ", precision * 100)
    print("False Discovery Rate: ", false_discovery_rate * 100)
    print("F1: ", f1 * 100)
          
    report = pd.DataFrame(columns=['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'False Discovery Rate', 'F1'], index=['Adaboost Train'])
    report.loc['Adaboost Train'] = [accuracy, sensitivity, specificity, precision, false_discovery_rate, f1]
    report.transpose()
    report.to_csv(f'./adaboost_{dataset_name}_report.csv', mode='a')



    

In [84]:
for i in range(1, 2):
    # run_adaboost('telco', i*5)
    # run_adaboost('credit', i*5)
    run_adaboost('adult', i*5)

There are missing values in the dataset.
Missing values per column:
workclass     1836
occupation    1843
dtype: int64
Missing values per column are fixed

There are missing values in the dataset.
Missing values per column:
workclass         963
occupation        966
native-country    274
dtype: int64
Missing values per column are fixed

k: 30
[ 4918  9874 27474 ... 12327  1255  4686]


100%|██████████| 1000/1000 [01:32<00:00, 10.77it/s]


y_pred: [1 0 0 ... 0 0 0]
[21964 24852 15932 ... 26731 21936 19252]


100%|██████████| 1000/1000 [01:43<00:00,  9.68it/s]


y_pred: [0 1 1 ... 1 0 1]
[ 6261 18902 20078 ... 12689 25532  3346]


...

y_pred: [1 1 0 ... 0 0 0]
[15568 27234 26632 ... 29222 22136 26821]


100%|██████████| 1000/1000 [01:35<00:00, 10.50it/s]


y_pred: [1 1 1 ... 0 1 1]
[24116 22312  9380 ... 30734 22235 14971]


100%|██████████| 1000/1000 [01:40<00:00,  9.97it/s]


y_pred: [1 0 0 ... 0 0 0]
alphas:  [0.33426966 0.2870561  0.13232518 0.14317351 0.10317555]
hyp 1: <class 'numpy.ndarray'>
hyp 2: numpy.ndarray[0]
Adaboost Test
Accuracy:  78.47183833916836
Sensitivity:  20.202808112324494
Specificity:  96.49376759147567
Precision:  64.0560593569662
False Discovery Rate:  35.9439406430338
F1:  30.717533109310136
hyp 1: <class 'numpy.ndarray'>
hyp 2: numpy.ndarray[0]
Adaboost Train
Accuracy:  78.17634593532141
Sensitivity:  19.971942354291546
Specificity:  96.6383495145631
Precision:  65.33166458072591
False Discovery Rate:  34.668335419274094
F1:  30.591912482906817


In [93]:

run_logistic_regression('adult')

There are missing values in the dataset.
Missing values per column:
workclass     1836
occupation    1843
dtype: int64
Missing values per column are fixed

There are missing values in the dataset.
Missing values per column:
workclass         963
occupation        966
native-country    274
dtype: int64
Missing values per column are fixed

k: 30


100%|██████████| 5000/5000 [00:16<00:00, 312.03it/s]


Logistic Regression Test
Accuracy:  84.85965235550643
Precision:  72.51385718943592
Recall:  57.8263130525221
F1:  64.34254303486185
